In [2]:
import math

In [3]:
input_size = 51
hidden_size = 50
block_size = 10

In [4]:
itos = {0: ' ',
 1: '&',
 2: "'",
 3: '(',
 4: ')',
 5: ',',
 6: '-',
 7: '.',
 8: '/',
 9: '0',
 10: '1',
 11: '2',
 12: '3',
 13: '4',
 14: '5',
 15: '6',
 16: '7',
 17: '8',
 18: '9',
 19: ':',
 20: ';',
 21: '[',
 22: ']',
 23: 'a',
 24: 'b',
 25: 'c',
 26: 'd',
 27: 'e',
 28: 'f',
 29: 'g',
 30: 'h',
 31: 'i',
 32: 'j',
 33: 'k',
 34: 'l',
 35: 'm',
 36: 'n',
 37: 'o',
 38: 'p',
 39: 'q',
 40: 'r',
 41: 's',
 42: 't',
 43: 'u',
 44: 'v',
 45: 'w',
 46: 'x',
 47: 'y',
 48: 'z',
 49: '|',
 50: '~'}

In [5]:
stoi = {' ': 0,
 '&': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '-': 6,
 '.': 7,
 '/': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 '6': 15,
 '7': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 ';': 20,
 '[': 21,
 ']': 22,
 'a': 23,
 'b': 24,
 'c': 25,
 'd': 26,
 'e': 27,
 'f': 28,
 'g': 29,
 'h': 30,
 'i': 31,
 'j': 32,
 'k': 33,
 'l': 34,
 'm': 35,
 'n': 36,
 'o': 37,
 'p': 38,
 'q': 39,
 'r': 40,
 's': 41,
 't': 42,
 'u': 43,
 'v': 44,
 'w': 45,
 'x': 46,
 'y': 47,
 'z': 48,
 '|': 49,
 '~': 50}

In [6]:
with open("/Users/nimitt/Documents/DigitalSystems/Project/Model_States/parameters_10_50_5000.txt",'r') as file: 
    params_data = file.read()

In [7]:
params_data = params_data.split('\n')

In [8]:
parameters = {'wf':[],'bf':[],'wi':[],'bi':[],'wc':[],'bc':[],'wo':[],'bo':[],'wy':[],'by':[]}
current_parameter = ''
for i in range(3, len(params_data)):
    if (params_data[i] == ''):
        continue
    if (params_data[i][:2] in parameters.keys()):
        current_parameter = params_data[i][:2]
        continue
    parameters[current_parameter].append(list(map(float,params_data[i].split())))

In [9]:
wf = parameters['wf']
bf = parameters['bf']
wi = parameters['wi']
bi= parameters['bi']
wc = parameters['wc']
bc= parameters['bc']
wo = parameters['wo']
bo = parameters['bo']
wy = parameters['wy']
by= parameters['by']

In [10]:
################# FPGA Forward ########################

def one_hot_encode(char):
    char_vector = [[0]]*input_size
    char_vector[stoi[char]] = [1]
    return char_vector

def one_hot_encode_text(text):
    text_mat = []
    for char in text:
        text_mat.append(one_hot_encode(char))
    return text_mat

def sigmoid(x):
    if (x <= -5):
        y = 0 
    elif ( -5<x <= -2):
        y = 0.0375*x + 0.194
    elif (-2<x<= 2):
        y =  0.190*x + 0.5
    elif (2<x<= 5):
        y =  0.0375*x + 0.805
    else:
        y = 1
    return y


def p_sigmoid(x):
    y = [[0]]*len(x)
    for i in range(len(x)):
        y[i] = [1/(1+math.exp(-1*x[i][0]))]
    # for i in range(len(x)):
    #     y[i] = [sigmoid(x[i][0])]
    return y

def p_tanh(x):
    y = [[0]]*len(x)

    for i in range(len(x)):
        y[i] = [(math.exp(x[i][0]) - math.exp(-1*x[i][0])) /(math.exp(x[i][0]) + math.exp(-1*x[i][0]))]
    # for i in range(len(x)):
    #     y[i] = [2*sigmoid(2*x[i][0]) - 1]
    return y

def matrix_into_vector(mat,vec):
    d = len(vec)
    m = len(mat[0])
    assert(d==m)
    n = len(mat)
    out = [[0]]*n
    for i in range(n):
        sum = 0
        for j in range(d):
            sum+=mat[i][j]*vec[j][0]
        out[i] = [sum]
    
    return out

def vector_plus_vector(vec1,vec2):
    for i in range(len(vec1)):
            vec1[i] = [vec1[i][0] + vec2[i][0]]
    return vec1

def vector_into_vector(vec1,vec2):
    out = [[0]]*len(vec1)
    for i in range(len(vec1)):
        out[i] = [vec1[i][0] * vec2[i][0]]
    return out

def forward( X):
    outputs = []
    seq_length = len(X)
    hidden_state = [[0]]*hidden_size
    cell_state = [[0]]*hidden_size

    for q in range(seq_length):
        concat_input = hidden_state + X[q]
        forget_gate = p_sigmoid(vector_plus_vector(matrix_into_vector(wf, concat_input), bf))
        input_gate = p_sigmoid(vector_plus_vector(matrix_into_vector(wi, concat_input), bi))
        candidate_gate = p_tanh(vector_plus_vector(matrix_into_vector(wc, concat_input), bc))
        output_gate = p_sigmoid(vector_plus_vector(matrix_into_vector(wo, concat_input), bo))
        cell_state = vector_plus_vector(vector_into_vector(forget_gate, cell_state), vector_into_vector(input_gate, candidate_gate))
        hidden_state = vector_into_vector(output_gate, p_tanh(cell_state))

        output = vector_plus_vector(matrix_into_vector(wy, hidden_state), by)
        outputs.append(output)
    return outputs

def one_hot_decode(y_hat):
    generated_char = ''
    max = 0
    for j in range(input_size):
        if y_hat[j][0] > y_hat[max][0]:
            max = j
    generated_char+=itos[max]

    return generated_char



In [11]:
############################## Testing #############################
prompt = 'this is '
max_len = 50

generated_text = '|'
for i in range(max_len):
    y_hat = forward(one_hot_encode_text(prompt))[-1]
    generated_char = one_hot_decode(y_hat)
    generated_text = generated_text + generated_char
    prompt = prompt + generated_char
generated_text+='|'
print(generated_text)


|affemint the industrial and emergences witho handh|


In [12]:
import struct

In [13]:
################ converting parameters to 32bit IEEE 754 #####################
def float_to_binary_IEEE754(f):
    packed = struct.pack('>f', f)
    binary = ''.join(format(byte, '08b') for byte in packed)
    
    return binary

In [14]:
parameters = [wf,bf,wi,bi,wo,bo,wc,bc,wy,by]
parameters_flatted = []
for parameter in parameters:
    for i in range(len(parameter)):
        for j in range(len(parameter[0])):
            parameters_flatted.append(float_to_binary_IEEE754(parameter[i][j]))


In [15]:
len(parameters_flatted)

23001

In [19]:
with open("/Users/nimitt/Documents/DigitalSystems/Project/Model_States/parameters_10_50_5000_ieee.txt",'w') as file:
    for i in range(len(parameters_flatted)):
        file.write(parameters_flatted[i])
        file.write(",")
        file.write('\n')